# Task 3

As I could not understand requirement of the task I will provide solution for some other problem.
I implemented KMEANS algorithm for three dimensions as R G B color presentation. It takes K as an parameter of color classes and divides colors into k groups and image after that will have k colors. Each pixel color is chosen as center color of the class it is in.

In [ ]:
import numpy as np
import pandas as pd


In [2]:
def EuclideanDistance(x, y):
    return np.sqrt((x-y)@(x-y).T)

In [3]:
def centersInit(x, k):
    centers = np.empty(shape=(k, 3))
    centers[0] = np.array(x.loc[np.random.choice(len(x), 1)[0]][['R', 'G', 'B']])
    points = np.array(x[(x.R != centers[0][0]) | (x.G != centers[0][1]) | (x.B != centers[0][2])][['R', 'G', 'B']])
    # Choose every next center such that it's min distance from already chosen points is maximum
    for chosen in range(1, k):
        p = []
        for point in points:
            distances = []
            for j in range(chosen):
                distances.append(EuclideanDistance(centers[j], point))
            p.append(min(distances))
        p = np.array(p)
        p = p / sum(p)
        centers[chosen] = points[np.random.choice(len(points), 1, p=p)][0]
        points = x[(x.R != centers[chosen][0]) | (x.G != centers[chosen][1]) | (x.B != centers[chosen][2])]
        chosen += 1

    return centers

In [4]:

def kMeans(x, k, epsilon=0.5):

    centers = centersInit(x, k)
    print(centers)
    while True:
        clusters = [] # list where clusters will be stored in
        for i in range(k):
            clusters.append([])

    #Choosing cluster for each point of X
        for _, point in x.T.iteritems():
            p = np.array(point[['R', 'G', 'B']])
            distances = []
            for j in range(k):
                distances.append(EuclideanDistance(centers[j], p))
            clusters[distances.index(min(distances))].append(point)
        #Updating each cluster's center with the mean of the cluster
        max_change = 0 #Maximum change of the center's position from its previous one
        for i in range(k):
            new_center: object = np.array([np.mean(np.array(clusters[i]).T[0]), np.mean(np.array(clusters[i]).T[1]), np.mean(np.array(clusters[i]).T[2])])
            max_change = max(max_change, EuclideanDistance(new_center, centers[i]))
            centers[i] = new_center

        if max_change < epsilon: #Stop the algorithm when the change is insignificant
            return clusters, centers

In [5]:
def encode(image):
    ret_dict = {}
    _r = []
    _g = []
    _b = []
    _c = []
    for h in range(image.shape[0]):
        for w in range(image.shape[1]):
            _r.append(image[h][w][0])
            _g.append(image[h][w][1])
            _b.append(image[h][w][2])
            _c.append([w, h])
    ret_dict['R'] = _r
    ret_dict['G'] = _g
    ret_dict['B'] = _b
    ret_dict['coordinate'] = _c
    return ret_dict


def decode(_dict, width, height):
    ret_img = np.zeros((width, height, 3,), dtype=uint8)
    for h in range(height):
        for w in range(width):
            ret_img[w][h][0] = _dict["R"][height*w + h]
            ret_img[w][h][1] = _dict["G"][height*w + h]
            ret_img[w][h][2] = _dict["B"][height*w + h]
    return ret_img

In [14]:
K=4
image = iio.v3.imread(".png") 
data = encode(image)
results = []


segment, colour = kMeans(data, K)
print(segment)
print(colour)
r = [np.array([colour[j][0] for i in range(len(segment[j]))]) for j in range(K)]
g = [np.array([colour[j][1] for i in range(len(segment[j]))]) for j in range(K)]
b = [np.array([colour[j][2] for i in range(len(segment[j]))]) for j in range(K)]
coord = [np.array([pixel['coordinate'] for pixel in seg]) for seg in segment]
R, G, B, C = r[0], g[0], b[0], coord[0]
for i in range(1, K):
    R = np.concatenate((R, r[i]))
    G = np.concatenate((G, g[i]))
    B = np.concatenate((B, b[i]))
    C = np.concatenate((C, coord[i]))

result = {'R': R, 'G': G, 'B': B, 'coordinate': C}
new_img = decode(result,img.shape[0],img.shape[1])

iio.imsave("img_new.jpg", new_img)

NameError: name 'iio' is not defined